# imports

In [1]:
from __future__ import division

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
from schlichtanders.mymatplotlib import Centre
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn
# plt.style.use('ggplot')

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
import os, platform, sys
from pprint import pprint

In [4]:
__file__ = os.path.realpath('__file__')
if platform.system() == "Windows":
    from schlichtanders.myos import replace_unc
    __file__ = replace_unc(__file__)
__path__ = os.path.dirname(__file__)
__parent__ = os.path.dirname(__path__)

In [46]:
from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float, Boolean
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker, Session, create_session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
import operator as op
from collections import defaultdict
import csv
import heapq
from copy import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
inf = float('inf')
from sklearn import cross_validation

In [47]:
import experiment_models
import experiment_util
from schlichtanders.mycontextmanagers import ignored
from schlichtanders.myobjects import Namespace
from evaluation import get_modes
import evaluation as eva

In [7]:
import numpy as np
import theano

In [8]:
model_names = { # sorted by optimization type
    "ml": ['baselinedet', 'baselinedetplus'],
    "annealing": ['baseline', 'baselineplus', 'mixture',
                  'planarflow', 'planarflowdet', 'radialflow', 'radialflowdet'],
    # first trials do not seem to be successfull, furthermore this needs a lot of time, maybe later on
    # "ml_exp_average": ['mixtureml', 'planarflowml', 'radialflowml'],
}
model_prefixes = reduce(op.add, model_names.values())

In [13]:
reload(experiment_util)

<module 'experiment_util' from '/home/stephan/GitProjects/theano_models/experiment/experiment_util.py'>

In [14]:
Hyper = experiment_util.get_old_hyper(model_prefixes)

# Dataset identifiers

In [69]:
datasetnames = ["boston", "concrete", "energy", "kin8nm", "powerplant", "winered", "yacht"]
datasetname = "boston"

data, error_func = experiment_util.load_and_preprocess_data(datasetname)
X, Z, VX, VZ, TX, TZ = data
example_input = X[0]
example_output = Z[0]
output_transfer = "softmax" if datasetname == "mnist" else "identity"
dataset_id = example_input, example_output, output_transfer

# Collect models and find best ones

In [37]:
reload(eva)

<module 'evaluation' from '/home/stephan/GitProjects/theano_models/experiment/__file__'>

In [38]:
n_best = 2
best_hyper = eva.get_best_hyper(["run_windows"], Hyper, model_prefixes,
                                n_best=n_best, test_suffix=["best_val_loss", "val_error_rate"],
                               key = lambda fn, p: datasetname in fn)

In [42]:
best_hyper

defaultdict(<function evaluation.defaultdictdict>,
            {'best_val_loss': defaultdict(dict,
                         {'baseline': {1: ([2.883494783484417,
                             3.218217335576597],
                            [hyper 8727366088249, hyper 8727366088281]),
                           2: ([2.4686542842699133, 2.727057631119438],
                            [hyper 8727365664697, hyper 8727365010213]),
                           3: ([2.67841449405836, 2.7496158682781715],
                            [hyper 8727365010325, hyper 8727365010309]),
                           4: ([4.1184674511785095, 4.21223083164381],
                            [hyper 8727365010301, hyper 8727365010285]),
                           8: ([2.0464404064676036, 2.4654262929722885],
                            [hyper 8727366088321, hyper 8727366417261]),
                           20: ([2.7365072499150815, 2.8814868650574614],
                            [hyper 8727365614365, hyper 8727365

In [50]:
best_hyper['best_val_loss']['baseline'][1][1][0].datasetname

u'boston'

# Compute Results on Test-set

In [68]:
reload(eva)
reload(experiment_models)

<module 'experiment_models' from '/home/stephan/GitProjects/theano_models/experiment/experiment_models.pyc'>

In [70]:
test_results = eva.compute_test_results(
    best_hyper, lambda hyper: experiment_util.load_and_preprocess_data(hyper.datasetname),
    model_module_id=dataset_id, n_trials=2
)

batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18

KeyboardInterrupt: 

# Sample the approximate posterior distribution for evaluation

In [89]:
reload(eva)

<module 'evaluation' from '/home/stephan/GitProjects/theano_models/experiment/__file__'>

In [90]:
best_hyper['best_val_loss']['planarflow'][2][1][0].planarflow_best_parameters

array([ 0.51119044, -0.95695661, -1.05459447, ...,  0.70644078,
        0.3328791 , -0.01993648])

In [91]:
best_hyper_samples = eva.sample_best_hyper(best_hyper, model_module_id=dataset_id)

## number of modes / histogram of modes

In [93]:
nr_modes = eva.get_nr_modes(best_hyper_samples)
nr_modes

defaultdict(<function evaluation.defaultdictdict>,
            {'best_val_loss': defaultdict(dict,
                         {'baseline': {1: Counter({1: 1501, 2: 1}),
                           2: Counter({1: 1501, 2: 1}),
                           3: Counter({1: 1502}),
                           4: Counter({1: 1502}),
                           8: Counter({1: 1502}),
                           20: Counter({1: 1502})},
                          'mixture': {1: Counter({1: 1502}),
                           2: Counter({1: 1501, 2: 1}),
                           3: Counter({1: 1474, 2: 24, 3: 4}),
                           4: Counter({1: 1459, 2: 43}),
                           8: Counter({1: 1497, 2: 5}),
                           20: Counter({1: 1477, 2: 24, 3: 1})},
                          'planarflow': {1: Counter({1: 1502}),
                           2: Counter({1: 1502}),
                           3: Counter({1: 1502}),
                           4: Counter({1: 1502}),
   

## Correlations between parameters

In [ ]:
reload(eva)

In [ ]:
correlations = eva.get_best_correlations(best_hyper_samples)

In [ ]:
"""
Bar chart demo with pairs of bars grouped for easy comparison.
"""
import numpy as np
import matplotlib.pyplot as plt


n_groups = 5

means_men = (20, 35, 30, 35, 27)
std_men = (2, 3, 4, 1, 2)

means_women = (25, 32, 34, 20, 25)
std_women = (3, 5, 2, 3, 3)

fig, ax = plt.subplots()

index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.4
error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, means_men, bar_width,
                 alpha=opacity,
                 color='b',
                 yerr=std_men,
                 error_kw=error_config,
                 label='Men')

rects2 = plt.bar(index + bar_width, means_women, bar_width,
                 alpha=opacity,
                 color='r',
                 yerr=std_women,
                 error_kw=error_config,
                 label='Women')

plt.xlabel('Group')
plt.ylabel('Scores')
plt.title('Scores by group and gender')
plt.xticks(index + bar_width, ('A', 'B', 'C', 'D', 'E'))
plt.legend()

plt.tight_layout()

In [ ]:
from itertools import count as icount

In [ ]:
1/(len(n_normflows) +1)

In [ ]:
fig, ax = plt.subplots()

index = np.arange(len(model_prefixes))
bar_width = 1/(len(n_normflows) +1)

opacity = 0.4
error_config = {'ecolor': '0.3'}
colors = ["b","g","r","c","m","y","k","w"]


for test in correlations:
    for i, nn, c in zip(icount(), n_normflows, colors):
        means = [abs(correlations[test][name][nn]).mean() for name in model_prefixes]
        stds = [abs(correlations[test][name][nn]).std() for name in model_prefixes]
        rect = plt.bar(index + i*bar_width, means, bar_width,
                     alpha=opacity,
                     color=c,
                     yerr=stds,
                     error_kw=error_config,
                     label='normflow=%i' % nn)

#     plt.xlabel('Group')
    plt.ylabel('absolute correlations')
    plt.title('Mean absolute correlations and standard deviations')
    plt.xticks(index + bar_width*(len(n_normflows)//2), model_prefixes)
    plt.legend()
    plt.tight_layout()

## KL divergence

This must be computed on histograms, as we need an approximation for the densities/pms

In [ ]:
reload(eva)

In [ ]:
best_kl = eva.compute_kl(best_hyper_samples)

In [ ]:
def add_colorbar(ax, h):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="10%", pad=0.15)
    cbar = plt.colorbar(h, cax=cax)
    
for nn in n_normflows:
    means = np.mean(best_kl['best_val_loss'][nn], axis=0)
    variances = np.var(best_kl['best_val_loss'][nn], axis=0)

    fig, axs = plt.subplots(ncols=2)
    h = axs[0].imshow(means, interpolation="None", cmap=plt.cm.RdBu_r, norm=Centre(0))
    axs[0].set_title("mean")
    add_colorbar(axs[0], h)

    h = axs[1].imshow(variances, interpolation="None", cmap=plt.cm.gray_r, norm=LogNorm())
    axs[1].set_title("variance")
    add_colorbar(axs[1], h)
    for ax in axs:
        ax.grid(False)
    fig.suptitle("#normflows = %i" %nn, fontsize=20, y=0.90)
    fig.tight_layout()
    # Make space for title
#     plt.subplots_adjust(top=0.85)